In [1]:
#!/usr/bin/env python3
import os
from pymongo import MongoClient

def get_mongo_client():
    """
    Retrieves MongoDB URI from the environment variable MONGO_URI.
    """
    uri = os.getenv("MONGO_URI")
    if not uri:
        raise RuntimeError("Please set the MONGO_URI environment variable.")
    return MongoClient(uri)

def find_eligible_content(max_auditors=5):
    """
    Connects to Tel_QA database and finds content IDs with fewer than
    max_auditors total (real audits + pending placeholders).
    """
    client = get_mongo_client()
    db = client["Tel_QA"]
    
    qa_col     = db["QA_pairs"]
    audit_col  = db["audit_logs"]
    assign_col = db["assignment_placeholders"]
    
    # All content IDs defined in QA_pairs
    content_ids = qa_col.distinct("content_id")
    
    eligible = []
    for cid in content_ids:
        real_count    = audit_col.count_documents({"content_id": cid})
        pending_count = assign_col.count_documents({"content_id": cid})
        total         = real_count + pending_count
        
        if total < max_auditors:
            eligible.append({
                "content_id": cid,
                "real_audits": real_count,
                "pending": pending_count,
                "total": total
            })
    
    return eligible

def main():
    eligible = find_eligible_content(max_auditors=5)
    if not eligible:
        print("✅ All content audited!")
    else:
        print("Content IDs still available for auditing (fewer than 5 assignments):")
        for item in eligible:
            print(f" - ID {item['content_id']}: {item['real_audits']} audits, "
                  f"{item['pending']} pending, total {item['total']}")

if __name__ == "__main__":
    main()


✅ All content audited!


In [5]:
#!/usr/bin/env python3
import os
from pymongo import MongoClient

def get_mongo_client():
    """
    Retrieves MongoDB URI from the environment variable MONGO_URI.
    """
    uri = os.getenv("MONGO_URI")
    if not uri:
        raise RuntimeError("Please set the MONGO_URI environment variable.")
    return MongoClient(uri)

def find_left_content(max_auditors=5):
    """
    Returns a list of dicts for each content_id that has been audited by
    fewer than `max_auditors` *distinct* interns.
    """
    client = get_mongo_client()
    db     = client["Tel_QA"]
    qa_col    = db["QA_pairs"]
    audit_col = db["audit_logs"]

    # 1) List all content IDs
    all_ids = qa_col.distinct("content_id")

    left = []
    for cid in all_ids:
        # 2) How many unique interns have audited this content?
        interns = audit_col.distinct("intern_id", {"content_id": cid})
        count   = len(interns)

        if count < max_auditors:
            left.append({
                "content_id":   cid,
                "distinct_audits": count
            })

    return left

def main():
    left = find_left_content(max_auditors=5)

    total_ids = len(left) + sum(1 for _ in find_left_content(max_auditors=0))  # hack to get total
    # Alternatively, fetch total separately:
    # total_ids = len(get_mongo_client()["Tel_QA"]["QA_pairs"].distinct("content_id"))

    if not left:
        print("✅ All content audited!")
    else:
        print(f"IDs Left: {len(left)} out of {total_ids}\n")
        for item in sorted(left, key=lambda x: x["distinct_audits"]):
            print(f" - ID {item['content_id']}: {item['distinct_audits']} distinct audits")

if __name__ == "__main__":
    main()


IDs Left: 116 out of 116

 - ID 14: 0 distinct audits
 - ID 28: 0 distinct audits
 - ID 38: 0 distinct audits
 - ID 60: 0 distinct audits
 - ID 98: 0 distinct audits
 - ID 166: 0 distinct audits
 - ID 177: 0 distinct audits
 - ID 181: 0 distinct audits
 - ID 249: 0 distinct audits
 - ID 257: 0 distinct audits
 - ID 282: 0 distinct audits
 - ID 287: 0 distinct audits
 - ID 303: 0 distinct audits
 - ID 309: 0 distinct audits
 - ID 310: 0 distinct audits
 - ID 315: 0 distinct audits
 - ID 318: 0 distinct audits
 - ID 326: 0 distinct audits
 - ID 327: 0 distinct audits
 - ID 336: 0 distinct audits
 - ID 338: 0 distinct audits
 - ID 348: 0 distinct audits
 - ID 355: 0 distinct audits
 - ID 363: 0 distinct audits
 - ID 374: 0 distinct audits
 - ID 384: 0 distinct audits
 - ID 401: 0 distinct audits
 - ID 402: 0 distinct audits
 - ID 406: 0 distinct audits
 - ID 412: 0 distinct audits
 - ID 413: 0 distinct audits
 - ID 415: 0 distinct audits
 - ID 425: 0 distinct audits
 - ID 427: 0 distinct 

In [3]:
#!/usr/bin/env python3
import os
from pymongo import MongoClient

def get_mongo_client():
    """
    Retrieves MongoDB URI from the environment variable MONGO_URI.
    """
    uri = os.getenv("MONGO_URI")
    if not uri:
        raise RuntimeError("Please set the MONGO_URI environment variable.")
    return MongoClient(uri)

def cleanup_over_audited(threshold=5):
    """
    Finds content_ids with more than `threshold` distinct interns in audit_logs,
    then removes all entries for those content_ids from both audit_logs and doubt_logs.
    Prints summary of actions taken.
    """
    client = get_mongo_client()
    db = client["Tel_QA"]
    audit_col = db["audit_logs"]
    doubt_col = db["doubt_logs"]

    # 1) Identify content_ids with > threshold distinct interns
    pipeline = [
        {"$group": {"_id": "$content_id", "interns": {"$addToSet": "$intern_id"}}},
        {"$project": {"count": {"$size": "$interns"}}},
        {"$match": {"count": {"$gt": threshold}}}
    ]
    over_audited = list(audit_col.aggregate(pipeline))
    content_ids = [doc["_id"] for doc in over_audited]
    count_ids = len(content_ids)

    print(f"Found {count_ids} content IDs audited by more than {threshold} interns:")
    print(content_ids)

    if count_ids == 0:
        print("No cleanup needed.")
        return

    # 2) Remove from audit_logs and doubt_logs
    res_audit = audit_col.delete_many({"content_id": {"$in": content_ids}})
    res_doubt = doubt_col.delete_many({"content_id": {"$in": content_ids}})

    print(f"Removed {res_audit.deleted_count} documents from 'audit_logs'.")
    print(f"Removed {res_doubt.deleted_count} documents from 'doubt_logs'.")

if __name__ == "__main__":
    cleanup_over_audited(threshold=5)


Found 17 content IDs audited by more than 5 interns:
[401, 514, 338, 60, 415, 177, 413, 287, 249, 511, 257, 14, 282, 573, 374, 166, 98]
Removed 618 documents from 'audit_logs'.
Removed 5 documents from 'doubt_logs'.


In [4]:
#!/usr/bin/env python3
import os
from pymongo import MongoClient

def get_mongo_client():
    """
    Retrieves MongoDB URI from the environment variable MONGO_URI.
    """
    uri = os.getenv("MONGO_URI")
    if not uri:
        raise RuntimeError("Please set the MONGO_URI environment variable.")
    return MongoClient(uri)

def cleanup_intern_logs(interns):
    """
    Finds content_ids that have any logs by the specified interns,
    then removes all entries for those content_ids from audit_logs,
    doubt_logs, and skipped_logs. Prints a summary.
    """
    client    = get_mongo_client()
    db        = client["Tel_QA"]
    audit_col = db["audit_logs"]
    doubt_col = db["doubt_logs"]
    skip_col  = db["skipped_logs"]

    # 1) Identify distinct content_ids logged by any of the interns
    content_ids = set()
    for col in (audit_col, doubt_col, skip_col):
        docs = col.distinct("content_id", {"intern_id": {"$in": interns}})
        content_ids.update(docs)

    count_ids = len(content_ids)
    print(f"Found {count_ids} distinct content IDs logged by interns {interns}:")
    print(sorted(content_ids))

    if count_ids == 0:
        print("No matching logs to remove.")
        return

    # 2) Remove from each collection
    res_audit = audit_col.delete_many({"content_id": {"$in": list(content_ids)}})
    res_doubt = doubt_col.delete_many({"content_id": {"$in": list(content_ids)}})
    res_skip  = skip_col.delete_many({"content_id": {"$in": list(content_ids)}})

    print(f"Removed {res_audit.deleted_count} documents from 'audit_logs'.")
    print(f"Removed {res_doubt.deleted_count} documents from 'doubt_logs'.")
    print(f"Removed {res_skip.deleted_count} documents from 'skipped_logs'.")

if __name__ == "__main__":
    # specify interns whose logs we want to remove
    interns_to_remove = ["katprx", "srimax", "aakhax"]
    cleanup_intern_logs(interns_to_remove)


Found 32 distinct content IDs logged by interns ['katprx', 'srimax', 'aakhax']:
[28, 38, 181, 303, 309, 310, 315, 318, 326, 327, 336, 348, 355, 363, 384, 402, 406, 412, 413, 425, 427, 428, 432, 438, 441, 443, 445, 461, 462, 472, 479, 552]
Removed 882 documents from 'audit_logs'.
Removed 6 documents from 'doubt_logs'.
Removed 33 documents from 'skipped_logs'.
